In [ ]:
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

import numpy
import numpy as np
import math
%matplotlib inline

In [ ]:
matplotlib.style.use('ggplot')
matplotlib.rcParams.update({'font.size': 12})
matplotlib.rcParams.update({'xtick.labelsize': 'x-large'})
matplotlib.rcParams.update({'xtick.major.size': '0'})
matplotlib.rcParams.update({'ytick.labelsize': 'x-large'})
matplotlib.rcParams.update({'ytick.major.size': '0'})
matplotlib.rcParams.update({"text.usetex": True})

In [ ]:
def f(theta, alpha, omega):
    val =  1 / (1 + np.exp( -omega * (np.cos(theta) - np.cos(alpha))))
    vmax = 1 / (1 + np.exp( -omega * (1 - np.cos(alpha))))
    vmin = 1 / (1 + np.exp( -omega * (-1 - np.cos(alpha))))
    return (val - vmin) / (vmax - vmin)

In [ ]:
xx = np.arange(-3.2,3.2,0.01)

fig,ax = plt.subplots()

oms = [2,5,10,20,50]

alpha = np.pi/3
for o in oms:
    ax.plot(xx, f(xx, alpha, o))

plt.legend([f"$\omega={o}$" for o in oms])

plt.setp(ax,
         xticks = [-np.pi,-2*alpha,-alpha,alpha,2*alpha,np.pi,0],
         xticklabels = ["$-\pi$","$-2\pi/3$","$-\pi/3$","$\pi/3$","$2\pi/3$","$\pi$",0],
         #ylabel = "$f$",
         title = r"Modulator $f(\theta, \alpha=\pi/3, \omega)$",
         xlabel = "$\\theta$" 
        )
plt.show(fig)
fig.savefig('../patchy-modulator.svg', bbox_inches='tight')


In [ ]:
greymap = matplotlib.colormaps['Greys']
norm = matplotlib.colors.Normalize(vmin=0, vmax=1)

bluemap = matplotlib.colormaps['Blues']
norm2 = matplotlib.colors.Normalize(vmin=0, vmax=1.4)

def draw_patchy_particle(ax, xy, director, alpha, omega, rcut):
    ax.set_axis_off()
    ax.set_aspect('equal')

    theta_director = numpy.rad2deg(numpy.arctan2(director[1], director[0]))

    extra = 30 # to capture the fade out of the patch
    n = 100
    dtheta = 2*(alpha+extra) / n
    for i in range(n):
        init_value = theta_director-alpha-extra
        offset = dtheta*i
        correction = 0.04
        colorscale = greymap(norm(f(np.deg2rad(-alpha-extra + dtheta * i), np.deg2rad(alpha), omega)))[0]
        wedge = matplotlib.patches.Wedge(
            xy,
            rcut,
            init_value + offset,
            init_value + offset + dtheta + correction, # dtheta is width of wedge, correction gets rid of hairline gaps between them
            alpha = 1 - colorscale,
            zorder=1,
            #color = bluemap(norm2(f(np.deg2rad(-alpha-extra + dtheta * i), np.deg2rad(alpha), omega))),
            linewidth=0 # to avoid edges overlapping
        )
        wedge = ax.add_patch(wedge)

    circle = matplotlib.patches.Circle(xy, rcut, zorder=2, ec='k',
                                      fill=False,
                                      linestyle=(5, (1, 8)))
    ax.add_patch(circle)

    unit_director = numpy.array(director) / numpy.linalg.norm(director)
    arrow = matplotlib.patches.FancyArrow(
        xy[0],
        xy[1],
        unit_director[0],#*rcut,
        unit_director[1],#*rcut,
        color='k',
        zorder=circle.get_zorder() + 1,
        width=0.03,
        length_includes_head=True,
        capstyle='projecting',
    )
    ax.add_patch(arrow)
    # adjust plot limits
    new_extent = rcut + max(abs(xy[0]), abs(xy[1]))
    old_extent = ax.get_xlim()[1]
    extent = max(new_extent, old_extent)
    ax.set_xlim(-extent, extent)
    ax.set_ylim(-extent, extent)

In [ ]:
arrowopts = dict(color = 'k', zorder = 3, width = 0.03, length_includes_head=True)

fig, ax = matplotlib.pyplot.subplots()

p1_x = -1.5
p2_x = 1.5
alpha = 70
omega = 10

draw_patchy_particle(ax, (p1_x,0), (1, 0.7), alpha, omega, 1.45)
draw_patchy_particle(ax, (p2_x,0), (-1, 0.4), alpha, omega, 1.45)

ax.add_patch(matplotlib.patches.FancyArrow(
    p1_x,
    0,
    p2_x-p1_x,
    0,
    capstyle='projecting',
    **arrowopts
))

theta_director = np.rad2deg(numpy.arctan2(0.7, 1))
ax.add_patch(matplotlib.patches.Arc([p1_x,0],1.3,1.3, theta1=0, theta2=theta_director, color='k', linewidth=1.5,))
ax.text(-0.7, 0.22, r'$\theta_i$', size=16, ha='center', va='center')

ax.add_patch(matplotlib.patches.Arc([p1_x,0],3,3, theta1=theta_director, theta2 = theta_director + alpha, color='k', linewidth=1.5, capstyle='projecting'))
ax.text(p1_x+0.9, -p1_x+0.02, r'$\alpha$', size=16, ha='center', va='center')

theta_director = np.rad2deg(numpy.arctan2(0.4, -1))
ax.add_patch(matplotlib.patches.Arc([p2_x,0],1.5,1.5, theta1=theta_director, theta2 = 180, color='k', linewidth=1.5))
ax.text(0.43, 0.22, r'$\theta_j$', size=16, ha='center', va='center')


ax.text(0.5, -0.4, r'$\vec{r}_{ij}$', size=16, color='k')
ax.text(1.1*p1_x, -0.3, r'$i$', size=16, ha='center', va='center')
ax.text(1.1*p2_x, -0.3, r'$j$', size=16, ha='center', va='center')

ax.set_ylim([-2,2])

# rot = np.deg2rad(30)
# lx = -2.5
# ly = -2.5
# ll = 1
# ax.add_line(matplotlib.lines.Line2D([lx,lx+ll*np.cos(rot)],[ly,ly+ll*np.cos(rot)],color='k',linewidth=0.5))
# #ax.text(ll-0.05,-0.1, "$x$")
# ax.add_line(matplotlib.lines.Line2D([lx,lx+ll*np.cos(rot+np.pi/2)],[ly,ly+ll*np.sin(rot+np.pi/2)],color='k',linewidth=0.5))
#ax.text(-0.1,ll-0.05, "$y$")

In [ ]:
fig.savefig('../patchy-pair.svg', bbox_inches='tight')

In [ ]:
arrowopts = dict(color = 'k', zorder = 3, width = 0.03, length_includes_head=True)

fig, ax = matplotlib.pyplot.subplots()

p1_x = 0
p2_x = 1.5
alpha = 70
omega = 10

rcut = 1.2
draw_patchy_particle(ax, (p1_x,0), (1, 0.7), alpha, omega, rcut)
#draw_patchy_particle(ax, (p2_x,0), (-1, 0.4), alpha, omega, 1.4)

# ax.add_patch(matplotlib.patches.FancyArrow(
#     p1_x,
#     0,
#     p2_x-p1_x,
#     0,
#     capstyle='projecting',
#     **arrowopts
# ))


ll = 1.4
ax.add_line(matplotlib.lines.Line2D([0,ll],[0,0],color='k',linewidth=0.5))
ax.text(ll-0.05,-0.1, "$x$")
ax.add_line(matplotlib.lines.Line2D([0,0],[0,ll],color='k',linewidth=0.5))
ax.text(-0.1,ll-0.05, "$y$")
ax.set_xlim([-ll,ll])
ax.set_ylim([-ll,ll])


theta_director = np.rad2deg(numpy.arctan2(0.7, 1))
# ax.add_patch(matplotlib.patches.Arc([p1_x,0],2,2, theta1=0, theta2=theta_director, color='k', linewidth=1.5,))
# ax.text(-0.7, 0.22, r'$\theta_i$', size=16, ha='center', va='center')

ax.add_patch(matplotlib.patches.Arc([p1_x,0],2.5,2.5, theta1=theta_director, theta2 = theta_director + alpha, color='k', linewidth=1.5, capstyle='projecting'))
ax.text(0.4, 1, r'$\alpha$', size=16, ha='center', va='center')

# theta_director = np.rad2deg(numpy.arctan2(0.4, -1))
# ax.add_patch(matplotlib.patches.Arc([p2_x,0],2,2, theta1=theta_director, theta2 = 180, color='k', linewidth=1.5))
# ax.text(0.32, 0.22, r'$\theta_j$', size=16, ha='center', va='center')

director = (1,0.7)
unit_director = numpy.array(director) / numpy.linalg.norm(director)


arrow = matplotlib.patches.FancyArrow(
    0,
    0,
    unit_director[0]*rcut,
    unit_director[1]*rcut,
    color='k',
    width=0.03,
    length_includes_head=True,
    capstyle='projecting',
)
# ax.add_patch(arrow)

ax.text(0.5, 0.2, r'$\vec{p}$', size=16, color='k')
# ax.text(1.1*p1_x, -0.3, r'$i$', size=16, ha='center', va='center')
# ax.text(1.1*p2_x, -0.3, r'$j$', size=16, ha='center', va='center')



In [ ]:
fig.savefig('../patchy-def.svg', bbox_inches='tight')